In [1]:
%%time
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
from dataclasses import dataclass
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv


@dataclass
class Config:
    project_dir: Path = Path("../../")
    model_dir: Path = project_dir / "models"
    outputs_dir: Path = project_dir / "outputs"
    data_dir: Path = project_dir / "data"
    raw_dir: Path = data_dir / "raw"
    interim_dir: Path = data_dir / "interim"
    processed_dir: Path = data_dir / "processed"


config = Config()
sys.path.append(str(config.project_dir.resolve()))
load_dotenv()

CPU times: user 666 ms, sys: 210 ms, total: 877 ms
Wall time: 1.44 s


True

In [2]:
df_orders = pd.read_csv(config.interim_dir / "olist_orders_merged.csv")

In [4]:
df_orders.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'approved_span', 'order_delivered_customer_date_span', 'is_delayed',
       'payment_type', 'payment_installments', 'payment_value',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',

In [31]:
tmp = df_orders.copy()
tmp["sum_price"] = tmp["price"] + tmp["freight_value"]

In [34]:
tmp[
    [
        "payment_value",
        "price",
        "freight_value",
        "sum_price",
        "payment_installments",
        "payment_type",
    ]
].query("payment_installments > 1").query("sum_price != payment_value")

,payment_value,price,freight_value,sum_price,payment_installments,payment_type
18,146.45,132.40,14.05,146.45,10.0,credit_card
20,43.09,27.99,15.10,43.09,4.0,credit_card
25,88.90,79.99,8.91,88.90,4.0,credit_card
30,227.68,90.90,48.64,139.54,10.0,credit_card
31,227.68,39.50,48.64,88.14,10.0,credit_card
...,...,...,...,...,...,...
116778,42.42,42.11,0.80,42.91,2.0,credit_card
116779,42.42,26.90,36.98,63.88,2.0,credit_card
116781,139.43,370.00,19.43,389.43,5.0,credit_card
116786,441.16,179.99,40.59,220.58,4.0,credit_card


In [36]:
tmp[
    [
        "payment_value",
        "price",
        "freight_value",
        "sum_price",
        "payment_installments",
        "payment_type",
    ]
].query("payment_installments > 1").query("sum_price == payment_value")

,payment_value,price,freight_value,sum_price,payment_installments,payment_type
3,179.12,159.90,19.22,179.12,3.0,credit_card
6,175.26,147.90,27.36,175.26,6.0,credit_card
8,75.16,59.99,15.17,75.16,3.0,credit_card
16,32.70,19.90,12.80,32.70,3.0,credit_card
19,1376.45,1299.00,77.45,1376.45,10.0,credit_card
...,...,...,...,...,...,...
116774,164.30,142.50,21.80,164.30,3.0,credit_card
116782,155.99,139.90,16.09,155.99,3.0,credit_card
116783,85.08,72.00,13.08,85.08,3.0,credit_card
116784,195.00,174.90,20.10,195.00,3.0,credit_card


In [40]:
tmp[
    [
        "payment_value",
        "price",
        "freight_value",
        "sum_price",
        "payment_installments",
        "payment_type",
    ]
].query("sum_price == payment_value").value_counts("payment_type")

payment_type
credit_card    53891
boleto         14206
voucher         1156
debit_card      1130
Name: count, dtype: int64

In [43]:
tmp[
    [
        "payment_value",
        "price",
        "freight_value",
        "sum_price",
        "payment_installments",
        "payment_type",
    ]
].query("sum_price != payment_value").value_counts("payment_type")

payment_type
credit_card    33725
boleto          8984
voucher         3116
debit_card       575
not_defined        3
Name: count, dtype: int64

In [47]:
tmp.query("order_id == '00143d0f86d6fbd9f9b38ab440ac16f5'")[
    [
        "order_id",
        "payment_value",
        "price",
        "freight_value",
        "sum_price",
        "payment_installments",
        "payment_type",
    ]
]

,order_id,payment_value,price,freight_value,sum_price,payment_installments,payment_type
42557,00143d0f86d6fbd9f9b38ab440ac16f5,109.29,21.33,15.1,36.43,1.0,credit_card
42558,00143d0f86d6fbd9f9b38ab440ac16f5,109.29,21.33,15.1,36.43,1.0,credit_card
42559,00143d0f86d6fbd9f9b38ab440ac16f5,109.29,21.33,15.1,36.43,1.0,credit_card


In [46]:
36.43 * 3

109.28999999999999

In [74]:
agg_price_by_order = (
    tmp.groupby("order_id")["sum_price"].sum().reset_index(name="sum_price_by_order")
)
agg_price_by_order["sum_price_by_order"] = agg_price_by_order[
    "sum_price_by_order"
].round(2)

In [82]:
agg_payment_value_by_order = (
    tmp[["order_id", "payment_value"]]
    .drop_duplicates()
    .groupby("order_id")["payment_value"]
    .sum()
    .reset_index(name="sum_payment_value_by_order")
)
agg_payment_value_by_order["sum_payment_value_by_order"] = agg_payment_value_by_order[
    "sum_payment_value_by_order"
].round(2)

In [83]:
pd.merge(
    agg_price_by_order, agg_payment_value_by_order, on="order_id", how="left"
).query("sum_price_by_order != sum_payment_value_by_order")

,order_id,sum_price_by_order,sum_payment_value_by_order
24,0010dedd556712d7bb69a19cb7bbd37a,0.00,111.12
36,0016dfedd97fc2950e388d2971d718c7,141.10,70.55
86,0035246a40f520710769010f752e7507,198.22,99.11
135,005d9a5423d47281ac463a968b3936fb,145.25,145.26
166,00789ce015e7e5791c7914f32bb4fad4,168.83,190.81
...,...,...,...
99264,ff850ba359507b996e8b2fbb26df8d03,66.02,33.01
99305,ffa1dd97810de91a03abd7bd76d2fed1,910.62,455.31
99309,ffa39020fe7c8a3e907320e1bec4b985,142.28,71.14
99320,ffaabba06c9d293a3c614e0515ddbabc,307.26,153.63
